In [3]:
pip install isic-cli

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import os
import cv2
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

def load_and_preprocess_data(image_dir, img_size=(224, 224)):
    images = []
    labels = []
    for class_name in os.listdir(image_dir):
        class_path = os.path.join(image_dir, class_name)
        if os.path.isdir(class_path):
            for image_name in os.listdir(class_path):
                img_path = os.path.join(class_path, image_name)
                try:
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, img_size)
                    img = img / 255.0
                    images.append(img)
                    labels.append(1 if class_name == 'malignant' else 0)
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

def augment_data():
    return ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        brightness_range=[0.8, 1.2]
    )

def create_model(input_shape=(224, 224, 3)):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate_model(model, train_generator, val_generator, batch_size=32, epochs=10):
    history = model.fit(train_generator, validation_data=val_generator, epochs=epochs)
    y_val = val_generator.classes
    y_pred = (model.predict(val_generator) > 0.5).astype("int32")
    print("Classification Report:")
    print(classification_report(y_val, y_pred))
    print(f"F1 Score: {f1_score(y_val, y_pred)}")
    print(f"Precision: {precision_score(y_val, y_pred)}")
    print(f"Recall: {recall_score(y_val, y_pred)}")
    return history

def save_model_and_results(model, history, output_dir="output"):
    os.makedirs(output_dir, exist_ok=True)
    model.save(os.path.join(output_dir, "model.h5"))
    with open(os.path.join(output_dir, "history.npy"), "wb") as f:
        np.save(f, history.history)

if __name__ == "__main__":
    image_dir = 'path_to_isic_dataset'
    batch_size = 32
    epochs = 10
    img_size = (224, 224)

    X, y = load_and_preprocess_data(image_dir, img_size=img_size)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)
    datagen = augment_data()

    train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)
    val_generator = ImageDataGenerator().flow(X_val, y_val, batch_size=batch_size)

    model = create_model(input_shape=(img_size[0], img_size[1], 3))
    history = train_and_evaluate_model(model, train_generator, val_generator, batch_size=batch_size, epochs=epochs)
    save_model_and_results(model, history)

ModuleNotFoundError: No module named 'tensorflow'